<a href="https://colab.research.google.com/github/jyothsna1076/AnamolyDetectionPRMLProject/blob/main/RandomForestPRML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.model_selection import train_test_split
np.random.seed(42)
df= pd.read_csv("Train_data.csv")
def le(df):
    for col in df.columns:
        if df[col].dtype == 'object':
                label_encoder = LabelEncoder()
                df[col] = label_encoder.fit_transform(df[col])

le(df)
x=  df.drop("class",axis= 1)
y= df["class"]
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size= 0.2, random_state= 42)


In [9]:
from collections import Counter

class MyRandomForest:
    def __init__(self, n_trees=10,sample_ratio=0.3, max_depth=None, n_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.sample_ratio = sample_ratio
        self.n_features = n_features
        self.trees = []

    def bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, int(n_samples* self.sample_ratio), replace=True)
        return X.iloc[indices], y.iloc[indices]

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            X_sample, y_sample = self.bootstrap_sample(X, y)
            tree = DecisionTreeClassifier(
                max_depth=self.max_depth,
                max_features=self.n_features,
                # random_state= 42
            )
            tree.fit(X_sample, y_sample)
            # plt.figure(figsize=(30, 20))
            # plot_tree(tree)
            # plt.show()
            self.trees.append(tree)

    def predict(self, X):
        tree_predictions = np.array([tree.predict(X) for tree in self.trees])
        final_predictions = []

        for sample_preds in tree_predictions.T:
            # Majority vote
            vote = Counter(sample_preds).most_common(1)[0][0]
            final_predictions.append(vote)

        return np.array(final_predictions)

In [10]:
import joblib
# best_sample, best_trees, best_depth = grid_search_rf(x, y, sample_ratios, n_trees_options, max_depths)
best_sample=0.5
best_trees= 15
best_depth = None
start_time = time.time()
final_model = MyRandomForest(n_trees=best_trees, sample_ratio=best_sample, max_depth=best_depth)
final_model.fit(x_train, y_train)
training_time = time.time() - start_time


joblib.dump(final_model, 'random_forset.pkl')
# print(f"Final Model Accuracy: {final_acc:.4f}")


['random_forset.pkl']

In [11]:
model =  joblib.load('random_forset.pkl')
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

# Print results
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"Training Time: {training_time:.2f} seconds")


Accuracy:  0.9980
Precision: 0.9980
Recall:    0.9980
F1 Score:  0.9980
Training Time: 0.84 seconds
